# Bermudan Max Call application

### Install packages which are not yet in TensorFlow Enterprise 2.15 (Intel® MKL-DNN/MK) environment

In [1]:
!pip install PyGithub
!pip install gurobipy

### CPU info of virtual machine

In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 25
model		: 17
model name	: AMD EPYC 9B14
stepping	: 1
microcode	: 0xffffffff
cpu MHz		: 2599.998
cache size	: 1024 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 16
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext invpcid_single ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves avx512_bf16 clzero xsaveerptr wbnoinvd arat avx512v

## Analysis Gradient norm tolerance parameter in Belomestny (2013)

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from BermudanMaxCall_AB_fullyLS import main as AndersonBroadie2004
from BermudanMaxCall_HK import main as Glassermann
from BermudanMaxCall_Belomestny import main as mainBelomestnyetal2009
from BermudanMaxCall_SAA import main as SAA_LP
from BermudanMaxCall_SAABelomestny import main as BelomestnySAA2013
from BermudanMaxCall_SchoenmakersSZH import main as SZH2013

import numpy as np
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 400000
traj_est_MM = 6800
traj_test_ub_desai2=30000

K_lower=400
traj_lb_testing = 100000
information=[]
steps=9


T=3
basis_f_K_U_MM=130
basis_f_K_U_MM_desai2=120
basis_f_K_U_MM_BHS=lambda n_stopping_rights: int(basis_f_K_U_MM*2) if n_stopping_rights>5 else (int(basis_f_K_U_MM*1.85) if n_stopping_rights==5 else  int(basis_f_K_U_MM*1.75))

K_L_basic= 250
dict_belom2009={10:87, 9: 103, 8: 125, 7:125, 6: 150, 5:155, 4:175,  3:189, 2: 220, 1:230}
K_U_belomestny2009=  lambda n_stopping_rights: dict_belom2009[n_stopping_rights]
dict_SZH={10: 100, 9:115, 8: 120, 7: 130, 6:150, 5: 160, 4:175, 3: 195, 2: 215, 1:235}
K_U_SZH2013=lambda n_stopping_rights: dict_SZH[n_stopping_rights]
K_L_AndersenBroadie2004={1:600, 2: 390, 3:350, 4:300, 5:300, 6:300, 7:300, 8:290}
K_L_Glasserman2004= {1: 400, 2: 410, 3: 360, 4: 295, 5: 285, 6:285, 7: 255, 8:255}
grid_Glasserman2004= lambda x: 1400 if x==1 else 900
inner_sim_SZH_= lambda n_stopping_rights:170+ 500//n_stopping_rights
inner_sim_AB= lambda x: 700 
fine_grid_belomestny2009_=lambda n_stopping_rights:150+ 500//n_stopping_rights if n_stopping_rights<8 else 200

traj_test_ub_AB=traj_test_ub//3
traj_lb_testing_GM=int(0.9*traj_test_ub)

r=0.0
delta_dividend=0.15

i=0
if __name__=='__main__': #(5,90,10),(5,90,10), (5,90,9), (5,90, 8),
    for d,s0,n_stopping_rights, steps, basis_f_K_U_MM_belomestny2013, traj_est_MM_belomestny2013 in [(5,90,1,9, 330, 15000), (5,90,1, 49, 60, 2000)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))              
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols= [d, s0, n_stopping_rights, '', '', '', '']


#             label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=SZH2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=inner_sim_SZH_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_SZH2013(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_(n_stopping_rights), info_cols=inf_cols)
#             information.append(inf_list)    

#             label_=f'Belom. et al. (2009) KU100-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=mainBelomestnyetal2009(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_belomestny2009(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_(n_stopping_rights), info_cols=inf_cols)
#             information.append(inf_list)   


#             K_L_AB=K_L_AndersenBroadie2004[n_stopping_rights] if n_stopping_rights in K_L_AndersenBroadie2004.keys() else np.min(list(K_L_AndersenBroadie2004.values()))
#             K_L_GM=K_L_Glasserman2004[n_stopping_rights] if n_stopping_rights in K_L_Glasserman2004.keys() else np.min(list(K_L_Glasserman2004.values())) 
#             label_ = f'Glasserman (2004)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=Glassermann(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid=grid_Glasserman2004(n_stopping_rights), K_low=K_L_GM,K_noise=None,  traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004(n_stopping_rights), info_cols=inf_cols)
#             information.append(inf_list)   

#             label_ = f'Anderson Broadie (2004)-{steps}'
#             print(''.join(np.repeat('-', 10)), label_, inner_sim_AB(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=AndersonBroadie2004(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid= inner_sim_AB(n_stopping_rights), K_low=K_L_AB, K_noise=None, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB(n_stopping_rights), info_cols=inf_cols)
#             information.append(inf_list)


#             label_ = f'Belomestny et al. (2019)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS(n_stopping_rights),  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9,  mode_desai_BBS_BHS='bhs')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) best-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) original-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012) high test-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  
            for g in [0.1, 0.001, 0.000001, 0.0000003]:
                for lambda_ in [1/20, 1/2, 1]:
                    label_ = f'Belomesty (2013); g_{g}; lambda={lambda_}'
                    print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
                    # if lambda_==1/20:
                    #     traj_est_MM_belomestny2013_=int(1.06*traj_est_MM_belomestny2013)
                    #     basis_f_K_U_MM_belomestny2013_=int(1.06*basis_f_K_U_MM_belomestny2013)
                    # elif lambda_==1/2:
                    #     traj_est_MM_belomestny2013_=int(1.02*traj_est_MM_belomestny2013)
                    #     basis_f_K_U_MM_belomestny2013_=int(1.02*basis_f_K_U_MM_belomestny2013)
                    # else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                    list_inf=BelomestnySAA2013(d, n_stopping_rights, True, g_tol=g, r=r, delta_dividend=delta_dividend,steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9, lambda_=lambda_, p=100)
                    inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                    information.append(inf_list) 
                    


                

with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
    pic.dump(information, fh)
repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

table_ = tabulate(utils.information_format(information), headers=utils.header_, tablefmt="latex_raw", floatfmt=".4f")

print(table_)

 

2024-06-20 21:11:00.741104: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 21:11:01.459802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 21:11:01.459873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 21:11:01.588829: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 21:11:01.841356: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Belomesty (2013); g_0.1; lambda=0.05 600 5 90 1 ----------
right= 0
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 0
Iteration: 0, Objective Value: 9.121197718768103   - Norm Jacobian 2.4471855566719207 - Max abs Jacobian 0.1719790830319748
Iteration: 10, Objective Value: 6.744310916870372   - Norm Jacobian 1.1450738949044166 - Max abs Jacobian 0.11062721766114868
Iteration: 20, Objective Value: 6.689185184873659   - Norm Jacobian 0.575501396096619 - Max abs Jacobian 0.06464055111075319
Iteration: 30, Objective Value: 6.6681092463357485   - Norm Jacobian 0.2469051554988719 - Max abs Jacobian 0.020260264555271128
Iteration: 40, Objective Value: 6.651828016012856   - Norm Jacobian 0.1678722868311191 - Max abs Jacobian 0.016451408435716013
Optimization terminated successfully.
         Current function value: 6.647666
         Iterations: 50
         Function evaluations: 54
         Gradient evaluations: 54
True
6.460546757420242
time solver [dat

### Run $\mathcal{N}_T =9$ (Rights 10 backwards to 1)

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from BermudanMaxCall_AB_fullyLS import main as AndersonBroadie2004
from BermudanMaxCall_HK import main as Glassermann
from BermudanMaxCall_Belomestny import main as mainBelomestnyetal2009
from BermudanMaxCall_SAA import main as SAA_LP
from BermudanMaxCall_SAABelomestny import main as BelomestnySAA2013
from BermudanMaxCall_SchoenmakersSZH import main as SZH2013

import numpy as np
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 400000
traj_est_MM = 6800
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
K_lower=400
traj_lb_testing = 100000
information=[]
steps=9


T=3
basis_f_K_U_MM=130
basis_f_K_U_MM_desai2=120
basis_f_K_U_MM_belomestny2013=330
basis_f_K_U_MM_BHS=lambda n_stopping_rights: int(basis_f_K_U_MM*2) if n_stopping_rights>5 else (int(basis_f_K_U_MM*1.85) if n_stopping_rights==5 else  int(basis_f_K_U_MM*1.75))

K_L_basic= 250
dict_belom2009={10:87, 9: 103, 8: 125, 7:125, 6: 150, 5:155, 4:175,  3:189, 2: 220, 1:230}
K_U_belomestny2009=  lambda n_stopping_rights: dict_belom2009[n_stopping_rights]
dict_SZH={10: 100, 9:115, 8: 120, 7: 130, 6:150, 5: 160, 4:175, 3: 195, 2: 215, 1:235}
K_U_SZH2013=lambda n_stopping_rights: dict_SZH[n_stopping_rights]
K_L_AndersenBroadie2004={1:600, 2: 390, 3:350, 4:300, 5:300, 6:300, 7:300, 8:290}
K_L_Glasserman2004= {1: 400, 2: 410, 3: 360, 4: 295, 5: 285, 6:285, 7: 255, 8:255}
grid_Glasserman2004= lambda x: 1400 if x==1 else 900
inner_sim_SZH_= lambda n_stopping_rights:170+ 500//n_stopping_rights
inner_sim_AB= lambda x: 700 
fine_grid_belomestny2009_=lambda n_stopping_rights:150+ 500//n_stopping_rights if n_stopping_rights<8 else 200

traj_test_ub_AB=traj_test_ub//3
traj_lb_testing_GM=int(0.9*traj_test_ub)

r=0.0
delta_dividend=0.15

i=0
if __name__=='__main__': #(5,90,10),(5,90,10), (5,90,9), (5,90, 8),
    for d,s0,n_stopping_rights in [(5,90,10), (5,90,9), (5,90,8), (5,90,7), (5,90,6), (5,90,5), (5,90,4), (5,90,3), (5,90,2), (5,90,1)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))
                      
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols= [d, s0, n_stopping_rights, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=inner_sim_SZH_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_SZH2013(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_belomestny2009(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004[n_stopping_rights] if n_stopping_rights in K_L_AndersenBroadie2004.keys() else np.min(list(K_L_AndersenBroadie2004.values()))
            K_L_GM=K_L_Glasserman2004[n_stopping_rights] if n_stopping_rights in K_L_Glasserman2004.keys() else np.min(list(K_L_Glasserman2004.values())) 
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid=grid_Glasserman2004(n_stopping_rights), K_low=K_L_GM,K_noise=None,  traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid= inner_sim_AB(n_stopping_rights), K_low=K_L_AB, K_noise=None, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS(n_stopping_rights),  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.06*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.06*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.02*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.02*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    table_ = tabulate(utils.information_format(information), headers=utils.header_, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


2024-06-21 21:19:57.514285: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 21:19:58.181049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 21:19:58.181125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 21:19:58.302654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 21:19:58.536294: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 220 5 90 10 ----------
right= 0
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 1
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 2
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 3
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 4
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 5
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 6
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 7
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 8
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 0
right= 1
right= 2
right= 3
right= 4
right= 5
right= 6
right= 7
right= 8
right= 9
right= 0
right= 1
right= 2
right= 3
right= 4
right= 5
right= 6
right= 7
right= 8
right= 9
right= 0
right= 1
right= 2
right= 3
right= 4
right= 5
right= 6
right= 7
right= 8
right= 9
Lowerbound
Value 30.375552215487122
Std 0.17305442961949777
Stop time-mean [0. 1. 2. 3. 4. 5. 

### d=10 run (higher dimensional problem) for L=3

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from BermudanMaxCall_AB_fullyLS import main as AndersonBroadie2004
from BermudanMaxCall_HK import main as Glassermann
from BermudanMaxCall_Belomestny import main as mainBelomestnyetal2009
from BermudanMaxCall_SAA import main as SAA_LP
from BermudanMaxCall_SAABelomestny import main as BelomestnySAA2013
from BermudanMaxCall_SchoenmakersSZH import main as SZH2013

import numpy as np
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 400000
traj_est_MM = 6800
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
K_lower=400
traj_lb_testing = 100000
information=[]
steps=9


T=3
basis_f_K_U_MM=130
basis_f_K_U_MM_desai2=120
basis_f_K_U_MM_belomestny2013=330
basis_f_K_U_MM_BHS=lambda n_stopping_rights: int(basis_f_K_U_MM*2) if n_stopping_rights>5 else (int(basis_f_K_U_MM*1.85) if n_stopping_rights==5 else  int(basis_f_K_U_MM*1.75))

K_L_basic= 250
dict_belom2009={10:87, 9: 103, 8: 125, 7:125, 6: 150, 5:155, 4:175,  3:189, 2: 220, 1:230}
K_U_belomestny2009=  lambda n_stopping_rights: dict_belom2009[n_stopping_rights]
dict_SZH={10: 100, 9:115, 8: 120, 7: 130, 6:150, 5: 160, 4:175, 3: 195, 2: 215, 1:235}
K_U_SZH2013=lambda n_stopping_rights: dict_SZH[n_stopping_rights]
K_L_AndersenBroadie2004={1:600, 2: 390, 3:350, 4:300, 5:300, 6:300, 7:300, 8:290}
K_L_Glasserman2004= {1: 400, 2: 410, 3: 360, 4: 295, 5: 285, 6:285, 7: 255, 8:255}
grid_Glasserman2004= lambda x: 1400 if x==1 else 900
inner_sim_SZH_= lambda n_stopping_rights:170+ 500//n_stopping_rights
inner_sim_AB= lambda x: 700 
fine_grid_belomestny2009_=lambda n_stopping_rights:150+ 500//n_stopping_rights if n_stopping_rights<8 else 200

traj_test_ub_AB=traj_test_ub//3
traj_lb_testing_GM=int(0.9*traj_test_ub)

delta_dividend=0.15
r=0.0

i=0
if __name__=='__main__': #(5,90,10),(5,90,10), (5,90,9), (5,90, 8),
    for d,s0,n_stopping_rights in [(10,90,3)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))
                          
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols= [d, s0, n_stopping_rights, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=inner_sim_SZH_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_SZH2013(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_belomestny2009(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004[n_stopping_rights] if n_stopping_rights in K_L_AndersenBroadie2004.keys() else np.min(list(K_L_AndersenBroadie2004.values()))
            K_L_GM=K_L_Glasserman2004[n_stopping_rights] if n_stopping_rights in K_L_Glasserman2004.keys() else np.min(list(K_L_Glasserman2004.values())) 
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid=grid_Glasserman2004(n_stopping_rights), K_low=K_L_GM,K_noise=None,  traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid= inner_sim_AB(n_stopping_rights), K_low=K_L_AB, K_noise=None, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS(n_stopping_rights),  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.06*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.06*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.02*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.02*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    table_ = tabulate(utils.information_format(information), headers=utils.header_, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


2024-06-22 19:13:08.100148: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 19:13:08.764122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 19:13:08.764195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 19:13:08.884717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-22 19:13:09.123582: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 336 10 90 3 ----------
right= 0
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 1
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 2
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 0
right= 1
right= 2
right= 0
right= 1
right= 2
right= 0
right= 1
right= 2
Lowerbound
Value 28.40094135151819
Std 0.09136837802842862
Stop time-mean [4.06152 5.9872  7.64134]
Stop time-std [2.0090832  1.97150606 1.78749629]
Upperbound
up 34.53411813700177
std 0.1692592521973565
CV est 28.241332223385026
CV std 0.17001132219815357
time avg training 0:03:15.450736
time avg ub 0:05:29.965021
---------- Belom. et al. (2009) KU100-9 316 10 90 3 ----------
right= 0
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 1
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 2
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 0
right= 1
right= 2
right= 0
right= 1
right= 2
right= 0
right= 1
right= 2
[[0.         0.13626821 0.07142775 0.0

### $\mathcal{N}_T =49$. Final run

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from BermudanMaxCall_AB_fullyLS import main as AndersonBroadie2004
from BermudanMaxCall_HK import main as Glassermann
from BermudanMaxCall_Belomestny import main as mainBelomestnyetal2009
from BermudanMaxCall_SAA import main as SAA_LP
from BermudanMaxCall_SAABelomestny import main as BelomestnySAA2013
from BermudanMaxCall_SchoenmakersSZH import main as SZH2013

import numpy as np
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 300000
traj_est_MM = 1500
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=2000
traj_lb_testing = 100000
information=[]
steps=49


T=3
basis_f_K_U_MM=25
basis_f_K_U_MM_desai2=18
basis_f_K_U_MM_belomestny2013=60
basis_f_K_U_MM_BHS=lambda n_stopping_rights: int(basis_f_K_U_MM*2) if n_stopping_rights>5 else (int(basis_f_K_U_MM*1.85) if n_stopping_rights==5 else  int(basis_f_K_U_MM*1.75))

K_L_basic= 150
dict_belom2009={3:70}
K_U_belomestny2009=  lambda n_stopping_rights: dict_belom2009[n_stopping_rights]
dict_SZH={3: 70}
K_U_SZH2013=lambda n_stopping_rights: dict_SZH[n_stopping_rights]
K_L_AndersenBroadie2004={3:20}
K_L_Glasserman2004= {3: 80}
grid_Glasserman2004= lambda x: 1150
inner_sim_SZH_= lambda n_stopping_rights:150
inner_sim_AB= lambda x: 580 
fine_grid_belomestny2009_=lambda n_stopping_rights:200

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=int(0.75*traj_test_ub)

delta_dividend=0.15
r=0.0

i=0
if __name__=='__main__': #(5,90,10),(5,90,10), (5,90,9), (5,90, 8),
    for d,s0,n_stopping_rights in [(5,90,3)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))
             
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols= [d, s0, n_stopping_rights, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, n_stopping_rights, True, r=r,delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=inner_sim_SZH_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_SZH2013(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, n_stopping_rights, True, r=r,delta_dividend=delta_dividend, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_(n_stopping_rights), K_low=K_L_basic,K_up=K_U_belomestny2009(n_stopping_rights), traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004[n_stopping_rights] if n_stopping_rights in K_L_AndersenBroadie2004.keys() else np.min(list(K_L_AndersenBroadie2004.values()))
            K_L_GM=K_L_Glasserman2004[n_stopping_rights] if n_stopping_rights in K_L_Glasserman2004.keys() else np.min(list(K_L_Glasserman2004.values())) 
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid=grid_Glasserman2004(n_stopping_rights), K_low=K_L_GM,K_noise=None,  traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB(n_stopping_rights), d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, steps=steps,T=T, grid= inner_sim_AB(n_stopping_rights), K_low=K_L_AB, K_noise=None, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB(n_stopping_rights), info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS(n_stopping_rights),  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, mode_kaggle=True, r=r, delta_dividend=delta_dividend, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r,delta_dividend=delta_dividend, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, S_0=s0, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  


            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, s0, n_stopping_rights, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.06*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.06*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.02*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.02*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, n_stopping_rights, True, r=r, delta_dividend=delta_dividend,steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, S_0=s0,  seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 



            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    table_ = tabulate(utils.information_format(information), headers=utils.header_, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)

2024-06-23 14:49:53.030509: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-23 14:49:53.665333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 14:49:53.665406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 14:49:53.780758: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 14:49:54.008400: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -49 150 5 90 3 ----------
right= 0
t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 1
t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 2
t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
right= 0
right= 1
right= 